# LightCSPNet on 38-Cloud Dataset
This notebook trains a basic CNN model for cloud segmentation using RGB input from the 38-Cloud dataset.

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [27]:
# Dataset for RGB input
class CloudDatasetRGB(Dataset):
    def __init__(self, red_dir, green_dir, blue_dir, mask_dir, transform=None):
        self.red_dir = red_dir
        self.green_dir = green_dir
        self.blue_dir = blue_dir
        self.mask_dir = mask_dir
        self.red_filenames = sorted(os.listdir(red_dir))
        self.transform = transform

    def __len__(self):
        return len(self.red_filenames)

    def __getitem__(self, idx):
        red_filename = self.red_filenames[idx]

        # Extract patch identifier (after "red_")
        patch_id = red_filename.replace("red_", "")

        # Construct green and blue filenames
        green_filename = "green_" + patch_id
        blue_filename = "blue_" + patch_id
        mask_filename = "gt_" + patch_id

        r = Image.open(os.path.join(self.red_dir, red_filename)).convert("L")
        g = Image.open(os.path.join(self.green_dir, green_filename)).convert("L")
        b = Image.open(os.path.join(self.blue_dir, blue_filename)).convert("L")
        rgb = Image.merge("RGB", (r, g, b))

        mask = Image.open(os.path.join(self.mask_dir, mask_filename)).convert("L")
        mask = np.array(mask)
        mask = (mask > 127).astype(np.float32)

        if self.transform:
            rgb = self.transform(rgb)
            mask = Image.fromarray(mask)
            mask = self.transform(mask)

        return rgb, mask

In [28]:
# LightCSPNet model
class LightCSPNet(nn.Module):
    def __init__(self):
        super(LightCSPNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 1, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.upsample = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)

    def forward(self, x):
        x1 = torch.relu(self.conv1(x))
        x2 = self.pool(torch.relu(self.conv2(x1)))
        x3 = self.pool(torch.relu(self.conv3(x2)))
        x4 = self.upsample(torch.relu(self.conv4(x3)))
        return torch.sigmoid(x4)

In [29]:
# Dice Coefficient
def dice_coeff(pred, target, threshold=0.5):
    pred = (pred > threshold).float()
    smooth = 1.0
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

In [30]:
# Training and evaluation functions
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for images, masks in dataloader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    dice_scores = []
    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            dice = dice_coeff(outputs, masks)
            dice_scores.append(dice.item())
    return np.mean(dice_scores)

In [31]:
# Set your dataset paths
red_dir = "38-Cloud_training/train_red"
green_dir = "38-Cloud_training/train_green"
blue_dir = "38-Cloud_training/train_blue"
mask_dir = "38-Cloud_training/train_gt"

# Image transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load dataset and dataloader
dataset = CloudDatasetRGB(red_dir, green_dir, blue_dir, mask_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [32]:
# Initialize model and train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LightCSPNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

for epoch in range(10):
    loss = train(model, dataloader, optimizer, criterion, device)
    dice = evaluate(model, dataloader, device)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Dice Coeff: {dice:.4f}")

Epoch 1, Loss: 0.6931, Dice Coeff: 0.0019
Epoch 2, Loss: 0.6931, Dice Coeff: 0.0010
Epoch 3, Loss: 0.6931, Dice Coeff: 0.0019
Epoch 4, Loss: 0.6931, Dice Coeff: 0.0013
Epoch 5, Loss: 0.6931, Dice Coeff: 0.0019
Epoch 6, Loss: 0.6931, Dice Coeff: 0.0000
Epoch 7, Loss: 0.6931, Dice Coeff: 0.0010
Epoch 8, Loss: 0.6931, Dice Coeff: 0.0000
Epoch 9, Loss: 0.6931, Dice Coeff: 0.0001
Epoch 10, Loss: 0.6931, Dice Coeff: 0.0000
